In [251]:
import pandas as pd
from glob import glob
import re
import warnings
warnings.filterwarnings('ignore')

In [252]:
dirr = 'C:/Users/shic/Desktop/shinhan_simon/google_drive/crawler_data/kbland/original/'

In [253]:
list_files = glob(dirr+'*.csv')
list_files

['C:/Users/shic/Desktop/shinhan_simon/google_drive/crawler_data/kbland/original\\매매가격_변화율_original.csv',
 'C:/Users/shic/Desktop/shinhan_simon/google_drive/crawler_data/kbland/original\\매매가격_지수_original.csv',
 'C:/Users/shic/Desktop/shinhan_simon/google_drive/crawler_data/kbland/original\\매매시장_매매거래_original.csv',
 'C:/Users/shic/Desktop/shinhan_simon/google_drive/crawler_data/kbland/original\\매매시장_매수우위_original.csv']

In [257]:
d1 = pd.read_csv(list_files[0],names=['year','month','region_b','region_s','ref_date','date_100','meme_rate','base_date'],header=0)
d2 = pd.read_csv(list_files[1],names=['year','month','region_b','region_s','ref_date','date_100','meme_jisu','base_date'],header=0)
d3 = pd.read_csv(list_files[2],names=['year','month','region_b','region_s','ref_date','date_100','market_meme','base_date'],header=0)
d4 = pd.read_csv(list_files[3],names=['year','month','region_b','region_s','ref_date','date_100','market_mesu','base_date'],header=0)

## 상단 그래프인 매매가격 그래프 데이터

In [258]:
# d2는 지수, d1은 변화율의 데이터입니다.
# 같은 그래프에 있는 데이터이나, 지수만 있고 변화율은 기록되지 않은 경우가 있습니다.
# 따라서 지수 데이터를 기준으로 join하고, 변화율이 기록되지 않은 NA에 대해서 0 처리해두었습니다.
df_a = pd.merge(d2.drop(['ref_date','date_100'],axis=1),
                d1.drop(['ref_date','date_100'],axis=1),how='left',on=['year','month','region_b','region_s','base_date'])

In [259]:
df_a.loc[df_a['meme_rate'].isna(),'meme_rate'] = 0
df_a['base_date'] = pd.to_datetime(df_a['base_date'],format='%Y,%m,%d')
df_a.drop(['year','month'],axis=1,inplace=True)
df_a['week_num'] = df_a['base_date'].dt.isocalendar().week
df_a = df_a[['base_date','region_b','region_s','week_num','meme_rate','meme_jisu']]
df_a

,base_date,region_b,region_s,week_num,meme_rate,meme_jisu
0,2011-02-28,서울,서울강북,9,0.05,82.2
1,2011-03-07,서울,서울강북,10,0.02,82.2
2,2011-03-14,서울,서울강북,11,0.02,82.3
3,2011-03-21,서울,서울강북,12,0.02,82.3
4,2011-03-28,서울,서울강북,13,0.00,82.3
...,...,...,...,...,...,...
867294,2021-05-31,제주/서귀포,제주/서귀포,22,0.55,101.8
867295,2021-06-07,제주/서귀포,제주/서귀포,23,0.71,102.6
867296,2021-06-14,제주/서귀포,제주/서귀포,24,0.91,103.5
867297,2021-06-21,제주/서귀포,제주/서귀포,25,0.61,104.1


## 하단 그래프인 매매시장 그래프 데이터

In [260]:
df_b = pd.merge(d3.drop(['ref_date','date_100'],axis=1),
                d4.drop(['ref_date','date_100'],axis=1),how='left',on=['year','month','region_b','region_s','base_date'])

In [261]:
df_b['base_date'] = pd.to_datetime(df_b['base_date'],format='%Y,%m,%d')
df_b.drop(['year','month'],axis=1,inplace=True)
df_b['week_num'] = df_b['base_date'].dt.isocalendar().week
df_b = df_b[['base_date','region_b','region_s','week_num','market_meme','market_mesu']]
df_b

,base_date,region_b,region_s,week_num,market_meme,market_mesu
0,2011-02-28,서울,서울강북,9,22.3,34.7
1,2011-03-07,서울,서울강북,10,16.5,32.3
2,2011-03-14,서울,서울강북,11,15.2,27.4
3,2011-03-21,서울,서울강북,12,11.0,26.8
4,2011-03-28,서울,서울강북,13,9.7,21.0
...,...,...,...,...,...,...
825090,2021-05-31,제주/서귀포,제주/서귀포,22,13.6,122.7
825091,2021-06-07,제주/서귀포,제주/서귀포,23,9.5,119.0
825092,2021-06-14,제주/서귀포,제주/서귀포,24,4.8,123.8
825093,2021-06-21,제주/서귀포,제주/서귀포,25,9.1,127.3


데이터 병합

In [262]:
df_bind = pd.merge(df_a,df_b,how='left',
                  on=['base_date','region_b','region_s','week_num'])

In [263]:
# 상단의 그래프(meme_rate,meme_jisu)는 있지만,
# 하단의 그래프(market_meme,market_mesu)는 없는 경우
df_bind.loc[df_bind['market_meme'].isna(),'market_meme'] = 0
df_bind.loc[df_bind['market_mesu'].isna(),'market_mesu'] = 0

지역 전처리

In [264]:
idx_1 = ['과천','광명','광주','구리','군포','김포','남양주','동두천','부천','시흥','안성','양주','오산','의왕','의정부','이천','파주','평택','하남','화성',
         '강릉','원주','춘천','제천','충주','계룡','공주','논산','당진','서산','아산','군산','익산','광양','목포','순천','여수','경산','구미','김천','안동',
         '거제', '김해','양산','진주','통영']
idx_2 = ['성남','수원','안산','안양','용인','청주','천안','전주','포항','창원']

In [265]:
df_bind['region_b'] = df_bind['region_b'].replace({
    '서울':'서울특별시','경기':'경기도','인천':'인천광역시','부산':'부산광역시','대구':'대구광역시',
    '광주':'광주광역시','대전':'대전광역시','울산':'울산광역시','강원':'강원도','충북':'충청북도',
    '충남':'충청남도','전북':'전라북도','전남':'전라남도','경북':'경상북도','경남':'경상남도',
    '세종':'세종특별자치시','제주/서귀포':'제주특별자치도'
})

In [266]:
df_bind['region_s'] = df_bind['region_s'].str.replace(r'서울|부산|대구|대전|울산|인천','')
df_bind.loc[df_bind['region_s']=='해운','region_s'] = '해운대구'
df_bind.loc[df_bind['region_b']=='광주광역시','region_s'] = df_bind.loc[df_bind['region_b']=='광주광역시','region_s'].str.replace('광주','')
df_bind.loc[df_bind['region_b']=='세종특별자치시','region_s'] = '세종특별자치시'
df_bind.loc[df_bind['region_b']=='제주특별자치도','region_s'] = '제주특별자치도'
df_bind.loc[df_bind['region_s'].isin(idx_1),'region_s'] = df_bind.loc[df_bind['region_s'].isin(idx_1),'region_s'] + '시'

for idxx in idx_2:
    df_bind['region_s'] = df_bind['region_s'].str.replace(idxx,idxx+'시 ')
df_bind['region_s'] = df_bind['region_s'].str.rstrip()

df_bind['base_date'] = df_bind['base_date'].astype(str).str.slice(stop=7).str.replace('-','')
df_bind

,base_date,region_b,region_s,week_num,meme_rate,meme_jisu,market_meme,market_mesu
0,201102,서울특별시,강북,9,0.05,82.2,22.3,34.7
1,201103,서울특별시,강북,10,0.02,82.2,16.5,32.3
2,201103,서울특별시,강북,11,0.02,82.3,15.2,27.4
3,201103,서울특별시,강북,12,0.02,82.3,11.0,26.8
4,201103,서울특별시,강북,13,0.00,82.3,9.7,21.0
...,...,...,...,...,...,...,...,...
8898960,202105,제주특별자치도,제주특별자치도,22,0.55,101.8,13.6,122.7
8898961,202106,제주특별자치도,제주특별자치도,23,0.71,102.6,9.5,119.0
8898962,202106,제주특별자치도,제주특별자치도,24,0.91,103.5,4.8,123.8
8898963,202106,제주특별자치도,제주특별자치도,25,0.61,104.1,9.1,127.3


In [267]:
df_bind.columns = ['BASE_YM','SIDONM','SIGNGU_NM','WK_NO','TRDE_PRICE_CHNG_RT','TRDE_PRICE_INDX','TRDE_DEAL_INDX','BUY_ADVAN_INDX']
df_bind

,BASE_YM,SIDONM,SIGNGU_NM,WK_NO,TRDE_PRICE_CHNG_RT,TRDE_PRICE_INDX,TRDE_DEAL_INDX,BUY_ADVAN_INDX
0,201102,서울특별시,강북,9,0.05,82.2,22.3,34.7
1,201103,서울특별시,강북,10,0.02,82.2,16.5,32.3
2,201103,서울특별시,강북,11,0.02,82.3,15.2,27.4
3,201103,서울특별시,강북,12,0.02,82.3,11.0,26.8
4,201103,서울특별시,강북,13,0.00,82.3,9.7,21.0
...,...,...,...,...,...,...,...,...
8898960,202105,제주특별자치도,제주특별자치도,22,0.55,101.8,13.6,122.7
8898961,202106,제주특별자치도,제주특별자치도,23,0.71,102.6,9.5,119.0
8898962,202106,제주특별자치도,제주특별자치도,24,0.91,103.5,4.8,123.8
8898963,202106,제주특별자치도,제주특별자치도,25,0.61,104.1,9.1,127.3


In [ ]:
df_bind = df_bind.groupby(['BASE_YM','SIDONM','SIGNGU_NM','WK_NO'],as_index=False).agg({
    'TRDE_PRICE_CHNG_RT' : 'mean',
    'TRDE_PRICE_INDX' : 'mean',
    'TRDE_DEAL_INDX' : 'mean',
    'BUY_ADVAN_INDX' : 'mean'
})
df_bind = df_bind.reset_index(drop=True)

In [268]:
df_bind.to_csv('C:/Users/shic/Desktop/shinhan_simon/google_drive/crawler_data/kbland/kbland_bind_preprocessed.csv',
               index=False,encoding='UTF-8-sig')